In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:

import pandas as pd
import json
import torch
import ast
from torch.utils.data import DataLoader
from transformers import DebertaTokenizer, DebertaForTokenClassification, AdamW
from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils.rnn as rnn_utils
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score



In [4]:
df=pd.read_csv('train.csv')
#print(type(train))
labels=[]
tokens=[]
print(df['span_start_index'][0][0])
for ind in df.index:
	label=[]

	span_start=json.loads(df['span_start_index'][ind])
	span_end=json.loads(df['span_end_index'][ind])
	token=ast.literal_eval((df['tokens'][ind]))
	tokens.append(token)
	for i in range(len(token)):
		flag=False
		for j in range(len(span_start)):
			if span_start[j]<i<=span_end[j]:
				label.append('I')
				flag=True
				continue
		if flag==True:
			continue
		if i in span_start:
			label.append('B')
		else:
			label.append('O')
	labels.append(label)

[


In [6]:
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')


 
label_dict = {'O':0,'B':1,'I':2}


int_labels = []

for i in labels:
	for j in range(len(i)):
		i[j]=label_dict[i[j]]
labelss=labels
#print(labels[0])

    
    


def convert_to_input(texts, labels):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    new_labels = []
    max_seq_len = max([len(seq) for seq in texts])
    
    for text, label in zip(texts, labels):
        tokens = []
        token_labels = []
        for word, lbl in zip(text, label):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            token_labels.extend([lbl] * len(word_tokens))
        
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        

        token_labels = [0] + token_labels + [0]
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        
        attention_mask=[]
        for i in range(len(input_id)):
            if token_labels[i]==0:
                attention_mask.append(0)
            else:
                attention_mask.append(1)
        #attention_mask = [1] * len(input_id)
        token_type_id = [0] * len(input_id)
        
        


        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        
        new_labels.append(token_labels)
      
    '''input_ids = pad_sequence([torch.tensor(ids) for ids in input_ids], batch_first=True,padding_value=0)
    attention_masks = pad_sequence([torch.tensor(mask) for mask in attention_masks], batch_first=True,padding_value=0)
    token_type_ids = pad_sequence([torch.tensor(ids) for ids in token_type_ids], batch_first=True,padding_value=0)
    padded_labels = []'''
    
    input_ids = pad_sequence([torch.tensor(ids, dtype=torch.long) for ids in input_ids], batch_first=True, padding_value=0)
    attention_masks = pad_sequence([torch.tensor(mask, dtype=torch.long) for mask in attention_masks], batch_first=True, padding_value=0)
    token_type_ids = pad_sequence([torch.tensor(ids, dtype=torch.long) for ids in token_type_ids], batch_first=True, padding_value=0)
    labels = pad_sequence([torch.tensor(lbls, dtype=torch.long) for lbls in new_labels], batch_first=True, padding_value=0)

    print(new_labels[0])
    
    
    #labels = pad_sequence([torch.tensor(lbls, dtype=torch.long) for lbls in new_labels], batch_first=True,padding_value=0)

    
    return input_ids, attention_masks, token_type_ids, labels
#print(len(tokens))
#print(len(labelss))
input_ids, attention_masks, token_type_ids, labels = convert_to_input(tokens, labelss)
print(len(input_ids))
print(len(attention_masks))
print(len(token_type_ids))
print(len(labels))




[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0]
6044
6044
6044
6044


'input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)\nattention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)\ntoken_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)\n\ninput_ids = torch.tensor(input_ids)\nattention_masks = torch.tensor(attention_masks)\ntoken_type_ids = torch.tensor(token_type_ids)\nlabels = []\nfor lbl in new_labels:\n    labels.extend(lbl)\nlabels = torch.tensor(labels)'

In [7]:
model = DebertaForTokenClassification.from_pretrained('microsoft/deberta-base', num_labels=3)  # Adjust num_labels according to your dataset
optimizer = AdamW(model.parameters(), lr=1e-4)
batch_size = 32
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks,labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    print("-" * 10)
    
    model.train()
    total_loss = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch
        
        
        
        
        if input_ids.size(0) != labels.size(0):
            continue
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    average_loss = total_loss / len(dataloader)
    print(f"Average Loss: {average_loss:.4f}")

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForTokenClassification: ['deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initial

Epoch 1/3
----------
Average Loss: 0.0209
Epoch 2/3
----------
Average Loss: 0.0041
Epoch 3/3
----------
Average Loss: 0.0025


In [8]:
model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits
        
        # Get predicted labels
        predicted_labels.extend(torch.argmax(logits, dim=2).tolist())
        true_labels.extend(labels.tolist())

# Flatten the predicted and true labels
predicted_labels = [label for sublist in predicted_labels for label in sublist]
true_labels = [label for sublist in true_labels for label in sublist]

# Compute metrics
report = classification_report(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='micro')
precision = precision_score(true_labels, predicted_labels, average='micro')
recall = recall_score(true_labels, predicted_labels, average='micro')

# Print metrics
print(report)
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    815558
           1       0.99      0.99      0.99     16208
           2       1.00      1.00      1.00     99010

    accuracy                           1.00    930776
   macro avg       1.00      1.00      1.00    930776
weighted avg       1.00      1.00      1.00    930776

F1 Score: 0.9997
Precision: 0.9997
Recall: 0.9997


In [9]:
df=pd.read_csv('dev.csv')
#print(type(train))
labels=[]
tokens=[]
print(df['span_start_index'][0][0])
for ind in df.index:
	label=[]

	span_start=json.loads(df['span_start_index'][ind])
	span_end=json.loads(df['span_end_index'][ind])
	token=ast.literal_eval((df['tokens'][ind]))
	tokens.append(token)
	for i in range(len(token)):
		flag=False
		for j in range(len(span_start)):
			if span_start[j]<i<=span_end[j]:
				label.append('I')
				flag=True
				continue
		if flag==True:
			continue
		if i in span_start:
			label.append('B')
		else:
			label.append('O')
	labels.append(label)

[


In [10]:
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')


# dictionary that maps integer to its string value 
label_dict = {'O':0,'B':1,'I':2}

# list to store integer labels 
int_labels = []

for i in labels:
	for j in range(len(i)):
		i[j]=label_dict[i[j]]
labelss=labels
#print(labels[0])
input_ids, attention_masks, token_type_ids, labels = convert_to_input(tokens, labelss)
batch_size = 32
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks,labels)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits
        
        # Get predicted labels
        predicted_labels.extend(torch.argmax(logits, dim=2).tolist())
        true_labels.extend(labels.tolist())

# Flatten the predicted and true labels
predicted_labels = [label for sublist in predicted_labels for label in sublist]
true_labels = [label for sublist in true_labels for label in sublist]

# Compute metrics
report = classification_report(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='micro')
precision = precision_score(true_labels, predicted_labels, average='micro')
recall = recall_score(true_labels, predicted_labels, average='micro')

# Print metrics
print(report)
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     69530
           1       0.98      0.96      0.97      2040
           2       0.99      1.00      1.00     12346

    accuracy                           1.00     83916
   macro avg       0.99      0.99      0.99     83916
weighted avg       1.00      1.00      1.00     83916

F1 Score: 0.9986
Precision: 0.9986
Recall: 0.9986
